In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/multi

In [ ]:
!pip install Flask-ngrok
from flask import Flask,render_template,url_for,request
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request, redirect, url_for
import os
import urllib3
import json
import base64
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import numpy as np

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/', methods=['GET', 'POST'])
def index(): # 기본화면 출력
    return render_template('home_3.html')

@app.route('/fileUpload', methods=['GET', 'POST'])
def file_upload(): # 이미지를 입력받아 저장 후에 버튼을 통해 view 호출
    global fdir
    global fn
    if request.method == 'GET':
        return False
    if request.method == 'POST':
        f = request.files['file']
        fn = f.filename[:-4]
        print(fn)
        fdir = './uploads/'
        f.save(fdir + fn + '.jpg')
        
        return render_template('home_4.html')

@app.route('/view', methods = ['GET', 'POST'])
def view(): # 이미지 처리 후 결과창을 호출
    
    search_object(fn, fdir)
    result = fn + 'fix.jpg'
    return render_template('result.html', result = result)

def search_object(fn, fdir): # AI API로 객체 정보와 정보에 따른 범위를 그려주고 이미지를 저장
    
    openApiURL = "http://aiopen.etri.re.kr:8000/ObjectDetect"
    accessKey = "99166867-3c08-41fd-a887-aeeef0289f89"          # 개인 인증키
    imageFilePath = fdir + fn + '.jpg'                           # 대상 이미지
    type = "jpg"

    
    file = open(imageFilePath, "rb")
    imageContents = base64.b64encode(file.read()).decode("utf8")
    file.close()
    

    requestJson = {
        "access_key": accessKey,
        "argument": {
            "type": type,
            "file": imageContents
        }
    }

    
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )
    
    json_data = json.loads(response.data)
    prn_data = json_data['return_object']

    
    df1 = pd.DataFrame(prn_data['data'])

    np.array(prn_data['data'], dtype='str')

    df1 = df1.astype({'confidence':'float', 'x':int, 'y':int, 'width':int, 'height':int})

    img = Image.open(fdir+fn +'.jpg')
    draw = ImageDraw.Draw(img)

    for i in range(len(df1)):
        x=df1.iloc[i, 2]
        y=df1.iloc[i, 3]
        w=df1.iloc[i, 4]
        h=df1.iloc[i, 5]

        if df1.iloc[i, 0] == 'person':
            color=(255,0,0)
        elif df1.iloc[i, 0] == 'car':
            color=(0,0,255)
        else:
            color = (0,255,0)
            
        draw.line([(x,y), (x,y+h), (x+w, y+h), (x+w, y), (x,y)], fill=color, width=2)
        draw.text((x, y-15), df1.iloc[i, 0], fill='red')

    img.save('./static/'+fn+'fix.jpg')


if __name__ == '__main__':
    app.run()